In [2]:
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel as fastapiBaseModel#이건 fastapi에서 사용하는 basemodel
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import Field
from langchain_core.pydantic_v1 import BaseModel as LangChainBaseModel#이건 아웃풋 고정할 때 사용하는 basemodel, 둘 다 있어야 함, 둘 다 이름 바꿔줘야 오류 안나니 주의
from langchain.memory import ChatMessageHistory
import os
from langchain_core.output_parsers import StrOutputParser
# from langchain.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import json

In [3]:
load_dotenv(dotenv_path="../.env")


True

In [4]:
name = ""
Gpersona = ""

def load_persona(session_id):#페르소나 로드
    global name
    global Gpersona
    if name==session_id:
        return Gpersona#같은 사람이랑 이야기할 때는 계속 페르소나를 읽지 않고 저장된 변수에서 가져오도록
    name = session_id
    persona_file = f"../Personas/persona_{session_id}.txt"
    if os.path.isfile(persona_file):
        with open(persona_file, 'r', encoding='utf-8') as f:
            persona = f.read()
            Gpersona = persona
    else:
        print(f"파일 '{persona_file}'이 존재하지 않습니다.")
        persona = "" 
    return persona
load_persona('Lily')

"\ufeff**Name:** Lily\\n\\n**Age:** 24\\n\\n**Gender:** Female\\n\\n**Vocation:** Farmer\\n\\n**Background:** Lily comes from a long lineage of farmers who have tilled the lands of this rural village for generations. Raised in a loving environment where the value of hard work and community was emphasized, she learned to work the fields and care for animals from an early age. After her parents' retirement, she took over the family farm, determined to uphold their legacy.\\n\\n**Looks:** Lily has a sun-kissed complexion with freckles lightly scattered across her nose and cheeks. Her bright green eyes are always full of warmth and cheer, highlighted by her wavy chestnut-brown hair, which she usually ties back in a practical ponytail or under a wide-brimmed hat. She often sports overalls with a floral print shirt and sturdy boots, embodying a blend of practicality and country charm.\\n\\n**Feature:** Despite her seemingly laid-back country appearance, Lily has an infectious smile that ligh

해당 장소에서 할 수 있는 활동들 목록 > 해당 장소의 사물 및 애니메이션에 따라 달라짐

In [5]:
house_activities = [
    "Cook a meal",
    "Watch TV",
    "Read a book",
    "Exercise",
    "Sleep",
    "Clean the house",
    "Work from home",
    "Call a friend",
    "Take a shower",
    "Use a desktop"
]
cafe_activities = [
    "Order coffee",
    "Chat with barista",
    "Use free Wi-Fi",
    "Read a magazine",
    "Work on laptop",
    "Meet a friend",
    "Listen to music",
    "People watch",
    "Sketch in a notebook",
    "Try a new pastry"
]
bakery_activities = [
    "Buy bread",
    "Purchase pastries",
    "Order a cake",
    "Chat with baker",
    "Sample free tasters",
    "Watch the baking process",
    "Learn a baking tip",
    "Order coffee",
    "Relax in seating area",
    "Sign up for baking class"
]
park_activities = [
    "Walk the dog",
    "Have a picnic",
    "Jog",
    "Read a book on a bench",
    "Feed the ducks",
    "Play frisbee",
    "Take photos",
    "Ride a bike",
    "Meditate",
    "Fly a kite"
]
convenience_store_activities = [
    "Buy snacks",
    "Purchase drinks",
    "Get lottery tickets",
    "Chat with cashier",
    "Pick up a magazine",
    "Use the ATM",
    "Get a hot dog",
    "Buy household items",
    "Refill phone credit",
    "Get ice cream"
]
bar_activities = [
    "Order a drink",
    "Play pool",
    "Listen to live music",
    "Watch a sports game",
    "Chat with bartender",
    "Dance",
    "Meet new people",
    "Sing karaoke",
    "Try a new cocktail",
    "Participate in trivia night"
]
library_activities = [
    "Borrow a book",
    "Study",
    "Use the computer",
    "Read in a quiet area",
    "Attend a workshop",
    "Join a book club",
    "Listen to an audiobook",
    "Research a topic",
    "Print documents",
    "Donate books"
]


인물의 페르소나에 맞춰 해당 인물의 일정 설계, 일정에는 공간과 무슨 행동을 할 지가 포함됨

In [6]:
persona_ryan = '''
 name : Ryan Park
 age : 29
 Innate tendency : imaginative, patient, kind
 Learned tendency : Tamara Taylor is a children's book author who loves to create stories that capture the imaginations of young readers. She is very patient and kind when it comes to her work.
 Currently : Ryan Park is working on a project to create a new mobile app. He is also reading up on the latest technologies to stay ahead of the curve.
 Lifestyle : Ryan Park goes to bed around 1am, awakes up around 9am, eats dinner around 5pm.
'''
persona_tamara = '''
 name : Tamara Taylor
 age : 30
 Innate tendency : analytical, pragmatic, driven
 Learned tendency : Ryan Park is a software engineer who loves to solve problems. He is constantly looking for ways to improve existing systems.
 Currently : Tamara Taylor lives with a housemate, Carmen Ortiz, and is working on a new series of children's books. She is also working on a comic book series for adults.
 Lifestyle : Tamara Taylor goes to bed around 10pm, awakes up around 6am, eats dinner around 6pm.
'''
persona_abigail = '''
 name : Abigail Chen
 age : 25
 Innate tendency : open-minded, curious, determined
 Learned tendency : Abigail Chen is a digital artist and animator who loves to explore how technology can be used to express ideas. She is always looking for new ways to combine art and technology.
 Currently : Abigail Chen is working on an animation project for a client. She is also experimenting with different tools and techniques to create interactive art.
 Lifestyle : Abigail Chen goes to bed around midnight, awakes up around 8am, eats dinner around 6pm.
'''
#당신은 상상력이 풍부한 게임 스토리 작가입니다. 요청에 답하라.
sysprom = "You are an imaginative game story writer. Answer the request."
#주어진 사람 {페르소나}의 페르소나를 이해합니다. 주변에는 테일러의 집, 카페, 베이커리, 공원, 편의점, 바, 도서관 등이 있습니다. 테일러의 집에서 할 수 있는 일은 {} 입니다. 카페에서 할 수 있는 일은 {} 입니다. 베이커리에서 할 수 있는 일은 {} 입니다. 공원에서 할 수 있는 일은 {} 입니다. 편의점에서 할 수 있는 일은 {} 입니다. 바에서 할 수 있는 일은 {} 입니다. 이해한 정보를 바탕으로 그 사람의 하루 일정을 계획합니다. 계획된 일정을 주어진 파이썬 목록의 형태로 출력합니다. {"hh:mm","place","활동","일정에 대한 자세한 설명"}
userprom = f"""Understand the persona of the given people, {persona_ryan},{persona_abigail},{persona_tamara}.
Around the people, there are places such as Ryan's house, Tamara's house, Abigail's house, cafe, bakery, park, convenience store, bar, and library.
What you can do at Ryan's house, Tamara's house, Abigail's house is {house_activities}. 
What you can do at a cafe is {cafe_activities}. 
What you can do at a bakery is {bakery_activities}. 
What you can do at a park is {park_activities}. 
What you can do at a convenience store is {convenience_store_activities}. 
What you can do at a bar is {bar_activities}. 
Plan each person's daily schedule based on the information you understand. 
Print out the planned schedule in the form of a given list. [['hh:mm','place','activity','Detailed description of the schedule']]"""
userprom = f"""Understand the persona of the given people, {persona_ryan},{persona_abigail},{persona_tamara}.
Around the people, there are places such as Ryan's house, Tamara's house, Abigail's house, cafe, bakery, park, convenience store, bar, and library.
What you can do at Ryan's house, Tamara's house, Abigail's house is {house_activities}. 
What you can do at a cafe is {cafe_activities}. 
What you can do at a bakery is {bakery_activities}. 
What you can do at a park is {park_activities}. 
What you can do at a convenience store is {convenience_store_activities}. 
What you can do at a bar is {bar_activities}. 
Plan each person's daily schedule based on the information you understand. 
Print out the planned schedule in list form in order of name, time, place, action, and explanation.
Print in the format given. tamara_schedule = [['06:00', 'place', 'activity', 'Detailed description of the schedule.'],...]
"""
userprom = f"""Understand the persona of the given people, {persona_ryan},{persona_abigail},{persona_tamara}.
Around the people, there are places such as Ryan's house, Tamara's house, Abigail's house, cafe, bakery, park, convenience store, bar, and library.
What you can do at Ryan's house, Tamara's house, Abigail's house is {house_activities}. 
What you can do at a cafe is {cafe_activities}. 
What you can do at a bakery is {bakery_activities}. 
What you can do at a park is {park_activities}. 
What you can do at a convenience store is {convenience_store_activities}. 
What you can do at a bar is {bar_activities}.  
Plan each person's daily schedule based on the information you understand. 
Output the planned schedule in json format given.
{'{'}
  "people": [
    {'{'}
      "name": "people's name",
      "schedule": [
        {'{'}
          "time": "hh:mm",
          "activity": "name of the activity",
          "place": "name of a place",
          "description": "Detailed description of the schedule."
        {'}'}
      ]
    {'}'}
  ]
{'}'}
"""
userprom = f"""Understand the persona of the given people, {persona_ryan},{persona_abigail},{persona_tamara}.
Around the people, there are places such as Ryan's house, Tamara's house, Abigail's house, cafe, bakery, park, convenience store, bar, and library.
What you can do at Ryan's house, Tamara's house, Abigail's house is {house_activities}. 
What you can do at a cafe is {cafe_activities}. 
What you can do at a bakery is {bakery_activities}. 
What you can do at a park is {park_activities}. 
What you can do at a convenience store is {convenience_store_activities}. 
What you can do at a bar is {bar_activities}.  
Plan each person's daily schedule based on the information you understand. 
Output the planned schedule in json format given.
[
  {'{'}
    "name": "people's name",
    "schedule": [
      {'{'}
        "time": "hh:mm",
        "activity": "name of the activity",
        "place": "name of a place",
        "description": "Detailed description of the schedule."
      {'}'}
    ]
  {'}'}
]
"""

In [7]:
print(userprom)

Understand the persona of the given people, 
 name : Ryan Park
 age : 29
 Innate tendency : imaginative, patient, kind
 Learned tendency : Tamara Taylor is a children's book author who loves to create stories that capture the imaginations of young readers. She is very patient and kind when it comes to her work.
 Currently : Ryan Park is working on a project to create a new mobile app. He is also reading up on the latest technologies to stay ahead of the curve.
 Lifestyle : Ryan Park goes to bed around 1am, awakes up around 9am, eats dinner around 5pm.
.
Around the people, there are places such as Ryan's house, Tamara's house, Abigail's house, cafe, bakery, park, convenience store, bar, and library.
What you can do at Ryan's house, Tamara's house, Abigail's house is ['Cook a meal', 'Watch TV', 'Read a book', 'Exercise', 'Sleep', 'Clean the house', 'Work from home', 'Call a friend', 'Take a shower', 'Use a desktop']. 
What you can do at a cafe is ['Order coffee', 'Chat with barista', 'Us

In [8]:
client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": sysprom},
        {"role": "user", "content": userprom}
    ],
    temperature=0.5,
    max_tokens=2000
)
print(completion.choices[0].message.content)

```json
[
  {
    "name": "Ryan Park",
    "schedule": [
      {
        "time": "09:00",
        "activity": "Wake up",
        "place": "Ryan's house",
        "description": "Ryan wakes up and starts his day with a refreshing shower."
      },
      {
        "time": "09:30",
        "activity": "Eat breakfast",
        "place": "Ryan's house",
        "description": "Ryan prepares a healthy breakfast to kickstart his day."
      },
      {
        "time": "10:00",
        "activity": "Work from home",
        "place": "Ryan's house",
        "description": "Ryan begins working on his mobile app project, focusing on coding and design."
      },
      {
        "time": "12:30",
        "activity": "Take a break",
        "place": "Ryan's house",
        "description": "Ryan takes a short break to stretch and relax."
      },
      {
        "time": "13:00",
        "activity": "Work from home",
        "place": "Ryan's house",
        "description": "Ryan continues working on his pro

In [ ]:
# client = OpenAI()
# completion = client.chat.completions.create(
#   model="gpt-4o",
#   messages=[
#     {"role": "system", "content": " "},
#     {"role": "user", "content": """

# """}
#   ]
# )
# completion.choices[0].message.content

In [ ]:
# print(completion.choices[0].message.content)

In [12]:
completion.choices[0].message.content

'```json\n[\n  {\n    "name": "Ryan Park",\n    "schedule": [\n      {\n        "time": "09:00",\n        "activity": "Wake up",\n        "place": "Ryan\'s house",\n        "description": "Ryan wakes up and starts his day with a refreshing shower."\n      },\n      {\n        "time": "09:30",\n        "activity": "Eat breakfast",\n        "place": "Ryan\'s house",\n        "description": "Ryan prepares a healthy breakfast to kickstart his day."\n      },\n      {\n        "time": "10:00",\n        "activity": "Work from home",\n        "place": "Ryan\'s house",\n        "description": "Ryan begins working on his mobile app project, focusing on coding and design."\n      },\n      {\n        "time": "12:30",\n        "activity": "Take a break",\n        "place": "Ryan\'s house",\n        "description": "Ryan takes a short break to stretch and relax."\n      },\n      {\n        "time": "13:00",\n        "activity": "Work from home",\n        "place": "Ryan\'s house",\n        "descripti

In [13]:
print(type(completion.choices[0].message.content))

<class 'str'>


In [1]:
import json

json_data = '```json\n[\n  {\n    "name": "Ryan Park",\n    "schedule": [\n      {\n        "time": "09:00",\n        "activity": "Wake up",\n        "place": "Ryan\'s house",\n        "description": "Ryan wakes up and starts his day with a refreshing shower."\n      },\n      {\n        "time": "09:30",\n        "activity": "Eat breakfast",\n        "place": "Ryan\'s house",\n        "description": "Ryan prepares a healthy breakfast to kickstart his day."\n      },\n      {\n        "time": "10:00",\n        "activity": "Work from home",\n        "place": "Ryan\'s house",\n        "description": "Ryan begins working on his mobile app project, focusing on coding and design."\n      },\n      {\n        "time": "12:30",\n        "activity": "Take a break",\n        "place": "Ryan\'s house",\n        "description": "Ryan takes a short break to stretch and relax."\n      },\n      {\n        "time": "13:00",\n        "activity": "Work from home",\n        "place": "Ryan\'s house",\n        "description": "Ryan continues working on his project, reading up on the latest technologies."\n      },\n      {\n        "time": "15:00",\n        "activity": "Order coffee",\n        "place": "Cafe",\n        "description": "Ryan heads to a nearby cafe to grab a coffee and take a break from work."\n      },\n      {\n        "time": "15:30",\n        "activity": "Work on laptop",\n        "place": "Cafe",\n        "description": "Ryan uses the cafe\'s free Wi-Fi to continue his work on the mobile app."\n      },\n      {\n        "time": "17:00",\n        "activity": "Eat dinner",\n        "place": "Ryan\'s house",\n        "description": "Ryan returns home and prepares dinner."\n      },\n      {\n        "time": "18:00",\n        "activity": "Exercise",\n        "place": "Park",\n        "description": "Ryan goes for a jog in the park to stay fit and healthy."\n      },\n      {\n        "time": "19:00",\n        "activity": "Read a book",\n        "place": "Library",\n        "description": "Ryan visits the library to read up on the latest technologies and trends."\n      },\n      {\n        "time": "21:00",\n        "activity": "Relax",\n        "place": "Ryan\'s house",\n        "description": "Ryan spends some time relaxing and unwinding at home."\n      },\n      {\n        "time": "22:00",\n        "activity": "Watch TV",\n        "place": "Ryan\'s house",\n        "description": "Ryan watches some TV to relax before bed."\n      },\n      {\n        "time": "00:00",\n        "activity": "Work from home",\n        "place": "Ryan\'s house",\n        "description": "Ryan does a bit more work on his project before calling it a night."\n      },\n      {\n        "time": "01:00",\n        "activity": "Sleep",\n        "place": "Ryan\'s house",\n        "description": "Ryan goes to bed to get some rest for the next day."\n      }\n    ]\n  },\n  {\n    "name": "Tamara Taylor",\n    "schedule": [\n      {\n        "time": "07:00",\n        "activity": "Wake up",\n        "place": "Tamara\'s house",\n        "description": "Tamara wakes up early to start her day."\n      },\n      {\n        "time": "07:30",\n        "activity": "Take a shower",\n        "place": "Tamara\'s house",\n        "description": "Tamara takes a refreshing shower to wake up fully."\n      },\n      {\n        "time": "08:00",\n        "activity": "Eat breakfast",\n        "place": "Tamara\'s house",\n        "description": "Tamara prepares a healthy breakfast to start her day right."\n      },\n      {\n        "time": "09:00",\n        "activity": "Work from home",\n        "place": "Tamara\'s house",\n        "description": "Tamara begins working on her latest children\'s book, creating imaginative stories."\n      },\n      {\n        "time": "12:00",\n        "activity": "Take a break",\n        "place": "Tamara\'s house",\n        "description": "Tamara takes a break to relax and recharge."\n      },\n      {\n        "time": "12:30",\n        "activity": "Order coffee",\n        "place": "Cafe",\n        "description": "Tamara heads to a nearby cafe to grab a coffee."\n      },\n      {\n        "time": "13:00",\n        "activity": "Read a magazine",\n        "place": "Cafe",\n        "description": "Tamara reads a magazine while enjoying her coffee."\n      },\n      {\n        "time": "14:00",\n        "activity": "Work from home",\n        "place": "Tamara\'s house",\n        "description": "Tamara continues working on her book, focusing on illustrations and story development."\n      },\n      {\n        "time": "17:00",\n        "activity": "Eat dinner",\n        "place": "Tamara\'s house",\n        "description": "Tamara prepares and eats dinner."\n      },\n      {\n        "time": "18:00",\n        "activity": "Go for a walk",\n        "place": "Park",\n        "description": "Tamara goes for a walk in the park to clear her mind and gather inspiration."\n      },\n      {\n        "time": "19:00",\n        "activity": "Read a book",\n        "place": "Tamara\'s house",\n        "description": "Tamara reads a book to unwind and gather more creative ideas."\n      },\n      {\n        "time": "21:00",\n        "activity": "Watch TV",\n        "place": "Tamara\'s house",\n        "description": "Tamara watches some TV to relax before bed."\n      },\n      {\n        "time": "22:30",\n        "activity": "Sleep",\n        "place": "Tamara\'s house",\n        "description": "Tamara goes to bed to get a good night\'s rest."\n      }\n    ]\n  }\n]\n```'

# Remove the triple backticks
json_data = json_data.strip('```')
json_data = json_data.strip('json')

# # Parse the string data into a Python list
# data = json.loads(json_data)
# jdata = json.dumps(data)

In [2]:
print(json_data)


[
  {
    "name": "Ryan Park",
    "schedule": [
      {
        "time": "09:00",
        "activity": "Wake up",
        "place": "Ryan's house",
        "description": "Ryan wakes up and starts his day with a refreshing shower."
      },
      {
        "time": "09:30",
        "activity": "Eat breakfast",
        "place": "Ryan's house",
        "description": "Ryan prepares a healthy breakfast to kickstart his day."
      },
      {
        "time": "10:00",
        "activity": "Work from home",
        "place": "Ryan's house",
        "description": "Ryan begins working on his mobile app project, focusing on coding and design."
      },
      {
        "time": "12:30",
        "activity": "Take a break",
        "place": "Ryan's house",
        "description": "Ryan takes a short break to stretch and relax."
      },
      {
        "time": "13:00",
        "activity": "Work from home",
        "place": "Ryan's house",
        "description": "Ryan continues working on his project, r

In [3]:
json_data

'\n[\n  {\n    "name": "Ryan Park",\n    "schedule": [\n      {\n        "time": "09:00",\n        "activity": "Wake up",\n        "place": "Ryan\'s house",\n        "description": "Ryan wakes up and starts his day with a refreshing shower."\n      },\n      {\n        "time": "09:30",\n        "activity": "Eat breakfast",\n        "place": "Ryan\'s house",\n        "description": "Ryan prepares a healthy breakfast to kickstart his day."\n      },\n      {\n        "time": "10:00",\n        "activity": "Work from home",\n        "place": "Ryan\'s house",\n        "description": "Ryan begins working on his mobile app project, focusing on coding and design."\n      },\n      {\n        "time": "12:30",\n        "activity": "Take a break",\n        "place": "Ryan\'s house",\n        "description": "Ryan takes a short break to stretch and relax."\n      },\n      {\n        "time": "13:00",\n        "activity": "Work from home",\n        "place": "Ryan\'s house",\n        "description": "R